In [2]:
%pip install xgboost xgboost_ray


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import json
import pandas as pd

import xgboost as xgb

import ray
ray.shutdown()

from ray import tune
from ray.air.checkpoint import Checkpoint
from ray.train.constants import TRAIN_DATASET_KEY

from ray.train.xgboost import XGBoostCheckpoint, XGBoostTrainer
from ray.air.config import ScalingConfig
from ray.data.preprocessor import Preprocessor

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [13]:
def ray_start_4_cpus():
    address_info = ray.init(num_cpus=4) #,log_dir='/tmp/ray_results/')
    yield address_info
    # The code after the yield will run as teardown code.
    ray.shutdown()

def ray_start_8_cpus():
    address_info = ray.init(num_cpus=8) #,log_dir='/tmp/ray_results/')
    yield address_info
    # The code after the yield will run as teardown code.
    ray.shutdown()

In [14]:
ray_start_8_cpus()

<generator object ray_start_8_cpus at 0x7ff3c88679d0>

In [15]:
data_raw = load_breast_cancer()

dataset_df = pd.DataFrame(data_raw["data"], columns=data_raw["feature_names"])
dataset_df["target"] = data_raw["target"]

train_df, test_df = train_test_split(dataset_df, test_size=0.3)

train_dataset = ray.data.from_pandas(train_df)
valid_dataset = ray.data.from_pandas(test_df)

In [20]:
import glob
from xgboost_ray import RayDMatrix, RayFileType

# We can also pass a list of files
path = list(sorted(glob.glob("./data/nyc-taxi/*.snappy.parquet")))

# This argument will be passed to `pd.read_parquet()`
columns = [
    "passenger_count",
    "trip_distance", "pickup_longitude", "pickup_latitude",
    "dropoff_longitude", "dropoff_latitude",
    "fare_amount", "extra", "mta_tax", "tip_amount",
    "tolls_amount", "total_amount"
]

dtrain = RayDMatrix(
    path,
    label="total_amount",  # Will select this column as the label
    columns=columns,
    # ignore=["total_amount"],  # Optional list of columns to ignore
    filetype=RayFileType.PARQUET)

IndexError: list index out of range

In [16]:
# def get_num_trees(booster: xgb.Booster) -> int:
#     data = [json.loads(d) for d in booster.get_dump(dump_format="json")]
#     return len(data)

In [17]:
# params = {
#     "tree_method": "approx",
#     "objective": "binary:logistic",
#     "eval_metric": ["logloss", "error"],
# }

hyperparameters = {
    "eta": "0.2",
    "gamma": "4",
    "max_depth": "5",
    "min_child_weight": "6",
    "num_round": "50",
    "objective": "reg:squarederror",
    "subsample": "0.7",
    "verbosity": "2",
    "content_type":"parquet",
}

scale_config = ScalingConfig(num_workers=4)

trainer = XGBoostTrainer(
    scaling_config=scale_config,
    label_column="target",
    params=params,
    datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
)
result = trainer.fit()

print(result)

# trainer = XGBoostTrainer(
#     scaling_config=ScalingConfig(
# #        trainer_resources={"CPU": 0},
#         num_workers=8,
# #        placement_strategy="SPREAD",
#         _max_cpu_fraction_per_node=0.9,
#     ),
#     label_column="target",
#     params=params,
#     datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
# )

# result = trainer.fit()

2023-03-10 03:41:40,853	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(XGBoostTrainer pid=11495) 2023-03-10 03:41:43,018	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[repartition]
(XGBoostTrainer pid=11495) 2023-03-10 03:41:44,516	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[repartition]
(XGBoostTrainer pid=11495) 2023-03-10 03:41:44,607	WARNING plan.py:524 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune
(XGBoostTrainer pid=11495) 2023-03-10 03:41:47,698	INFO tracker.py:217 -- start listen on 169.255.255.2:47043
(_RemoteRayXGBoostActor pid=11559) [03:41:47] task [xgboost.ray]:139674558309392 got new rank 0
(_RemoteRayXGBoostActor pid=11560) [03:41:47] task [xgboost.ray]:140514935999824 got new rank 1
(_Remo

Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train-error,train-logloss,training_iteration,trial_id,valid-error,valid-logloss,warmup_time
XGBoostTrainer_7809e_00000,2023-03-10_03-41-51,True,,a205b4d618b14a7aa7ec5a0ffd9dafdd,0,datascience-1-0-ml-m5-2xlarge-edc54522b6c2075d0551eb4e13d2,11,169.255.255.2,11495,True,8.27039,0.0733163,8.27039,1678419711,0,,0.00505051,0.054291,11,7809e_00000,0.047619,0.121991,0.00598192


(XGBoostTrainer pid=11495) 2023-03-10 03:41:51,007	INFO tracker.py:387 -- @tracker All nodes finishes job
2023-03-10 03:41:51,315	INFO tune.py:799 -- Total run time: 10.46 seconds (10.42 seconds for the tuning loop).


Result(metrics={'train-logloss': 0.05429103939953928, 'train-error': 0.00505050505050505, 'valid-logloss': 0.1219907808339312, 'valid-error': 0.04761904761904762, 'should_checkpoint': True, 'done': True, 'trial_id': '7809e_00000', 'experiment_tag': '0'}, error=None, log_dir=PosixPath('/root/ray_results/XGBoostTrainer_2023-03-10_03-41-40/XGBoostTrainer_7809e_00000_0_2023-03-10_03-41-40'))


# Resume from checkpoint

In [9]:
# trainer = XGBoostTrainer(
#     scaling_config=scale_config,
#     label_column="target",
#     params=params,
#     num_boost_round=5,
#     datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
# )
# result = trainer.fit()
# checkpoint = XGBoostCheckpoint.from_checkpoint(result.checkpoint)
# xgb_model = checkpoint.get_model()
# assert get_num_trees(xgb_model) == 5

2023-03-10 03:38:19,263	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(XGBoostTrainer pid=7544) 2023-03-10 03:38:21,945	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[repartition]
(XGBoostTrainer pid=7544) 2023-03-10 03:38:23,441	INFO bulk_executor.py:39 -- Executing DAG InputDataBuffer[Input] -> AllToAllOperator[repartition]
(XGBoostTrainer pid=7544) 2023-03-10 03:38:26,698	INFO tracker.py:217 -- start listen on 169.255.255.2:49487
(_RemoteRayXGBoostActor pid=7610) [03:38:26] task [xgboost.ray]:139796614291600 got new rank 0
(_RemoteRayXGBoostActor pid=7611) [03:38:26] task [xgboost.ray]:140051997375568 got new rank 1
(_RemoteRayXGBoostActor pid=7612) [03:38:27] task [xgboost.ray]:139786840095120 got new rank 2
(XGBoostTrainer pid=7544) 2023-03-10 03:38:27,161	INFO tracker.py:381 -- @tracker All of 4 nodes getting started
(_RemoteRayXGBoostActor pid=7613) [03:38:27] task [xgboost.ray]:140119573118160 got new rank 3


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train-error,train-logloss,training_iteration,trial_id,valid-error,valid-logloss,warmup_time
XGBoostTrainer_ffe1b_00000,2023-03-10_03-38-29,True,,a9bde25d93194f1ba5826af5254e9902,0,datascience-1-0-ml-m5-2xlarge-edc54522b6c2075d0551eb4e13d2,6,169.255.255.2,7544,True,7.91851,0.311455,7.91851,1678419509,0,,0.00505051,0.149229,6,ffe1b_00000,0.0654762,0.217007,0.00587487


(XGBoostTrainer pid=7544) 2023-03-10 03:38:29,393	INFO tracker.py:387 -- @tracker All nodes finishes job
2023-03-10 03:38:29,903	INFO tune.py:799 -- Total run time: 10.64 seconds (10.60 seconds for the tuning loop).


NameError: name 'get_num_trees' is not defined

# Tuner

In [10]:
trainer = XGBoostTrainer(
    scaling_config=scale_config,
    label_column="target",
    params={**params, **{"max_depth": 1}},
    datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
)

tune.run(
    trainer.as_trainable(),
    config={"params": {"max_depth": tune.randint(2, 4)}},
    num_samples=2,
)

# # Make sure original Trainer is not affected.
# assert trainer.params["max_depth"] == 1

2023-03-10 03:39:24,364	WARNING callback.py:109 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(XGBoostTrainer pid=8617) 2023-03-10 03:39:26,961	WARNING plan.py:524 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune
(XGBoostTrainer pid=8617) 2023-03-10 03:39:26,962	WARNING plan.py:524 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason is that cluster resources are used by Actors or Tune trials; see the following link for more details: https://docs.ray.io/en/master/data/dataset-internals.html#datasets-and-tune
(XGBoostTrainer pid=8617) 2023-03-10 03:39:26,996	WARNING plan.py:524 -- Warning: The Ray cluster currently does not have any available CPUs. The Dataset job will hang unless more CPUs are freed up. A common reason

(autoscaler +1m37s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +1m37s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.
(autoscaler +2m12s) Warning: The following resource request cannot be scheduled right now: {'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


2023-03-10 03:40:19,166	WARNING tune.py:147 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-10 03:40:22,198	ERROR tune.py:794 -- Trials did not complete: [XGBoostTrainer_26aea_00000, XGBoostTrainer_26aea_00001]
2023-03-10 03:40:22,199	INFO tune.py:799 -- Total run time: 57.84 seconds (57.79 seconds for the tuning loop).
2023-03-10 03:40:22,200	WARNING tune.py:805 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [ ]:
# # Move checkpoint to a different directory.
# checkpoint_dict = result.checkpoint.to_dict()
# checkpoint = Checkpoint.from_dict(checkpoint_dict)
# checkpoint_path = checkpoint.to_directory(tmpdir)
# resume_from = Checkpoint.from_directory(checkpoint_path)

# trainer = XGBoostTrainer(
#     scaling_config=scale_config,
#     label_column="target",
#     params=params,
#     num_boost_round=5,
#     datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     resume_from_checkpoint=resume_from,
# )
# result = trainer.fit()
# checkpoint = XGBoostCheckpoint.from_checkpoint(result.checkpoint)
# model = checkpoint.get_model()
# assert get_num_trees(model) == 10


In [ ]:
# @pytest.mark.parametrize(
#     "freq_end_expected",
#     [
#         (4, True, 7),  # 4, 8, 12, 16, 20, 24, 25
#         (4, False, 6),  # 4, 8, 12, 16, 20, 24
#         (5, True, 5),  # 5, 10, 15, 20, 25
#         (0, True, 1),
#         (0, False, 0),
#     ],
# )
# def test_checkpoint_freq(ray_start_4_cpus, freq_end_expected):
#     freq, end, expected = freq_end_expected

#     train_dataset = ray.data.from_pandas(train_df)
#     valid_dataset = ray.data.from_pandas(test_df)
#     trainer = XGBoostTrainer(
#         run_config=ray.air.RunConfig(
#             checkpoint_config=ray.air.CheckpointConfig(
#                 checkpoint_frequency=freq, checkpoint_at_end=end
#             )
#         ),
#         scaling_config=scale_config,
#         label_column="target",
#         params=params,
#         num_boost_round=25,
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     )
#     result = trainer.fit()

#     # Assert number of checkpoints
#     assert len(result.best_checkpoints) == expected, str(
#         [
#             (metrics["training_iteration"], _cp._local_path)
#             for _cp, metrics in result.best_checkpoints
#         ]
#     )

#     # Assert checkpoint numbers are increasing
#     cp_paths = [cp._local_path for cp, _ in result.best_checkpoints]
#     assert cp_paths == sorted(cp_paths), str(cp_paths)


# def test_preprocessor_in_checkpoint(ray_start_4_cpus, tmpdir):
#     train_dataset = ray.data.from_pandas(train_df)
#     valid_dataset = ray.data.from_pandas(test_df)

#     class DummyPreprocessor(Preprocessor):
#         def __init__(self):
#             super().__init__()
#             self.is_same = True

#         def fit(self, dataset):
#             self.fitted_ = True

#         def _transform_pandas(self, df: "pd.DataFrame") -> "pd.DataFrame":
#             return df

#     trainer = XGBoostTrainer(
#         scaling_config=scale_config,
#         label_column="target",
#         params=params,
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#         preprocessor=DummyPreprocessor(),
#     )
#     result = trainer.fit()

#     # Move checkpoint to a different directory.
#     checkpoint_dict = result.checkpoint.to_dict()
#     checkpoint = Checkpoint.from_dict(checkpoint_dict)
#     checkpoint_path = checkpoint.to_directory(tmpdir)
#     resume_from = Checkpoint.from_directory(checkpoint_path)

#     resume_from = XGBoostCheckpoint.from_checkpoint(resume_from)

#     model, preprocessor = resume_from.get_model(), resume_from.get_preprocessor()
#     assert get_num_trees(model) == 10
#     assert preprocessor.is_same
#     assert preprocessor.fitted_




In [ ]:

# train_dataset = ray.data.from_pandas(train_df)
# valid_dataset = ray.data.from_pandas(test_df)
# #with pytest.raises(KeyError, match=TRAIN_DATASET_KEY):
# XGBoostTrainer(
#     scaling_config=ScalingConfig(num_workers=2),
#     label_column="target",
#     params=params,
#     datasets={"valid": valid_dataset},
# )
# with pytest.raises(KeyError, match="dmatrix_params"):
#     XGBoostTrainer(
#         scaling_config=ScalingConfig(num_workers=2),
#         label_column="target",
#         params=params,
#         dmatrix_params={"data": {}},
#         datasets={TRAIN_DATASET_KEY: train_dataset, "valid": valid_dataset},
#     )




In [ ]:
# def test_distributed_data_loading(ray_start_4_cpus):
#     """Checks that XGBoostTrainer does distributed data loading for Ray Datasets."""

#     class DummyXGBoostTrainer(XGBoostTrainer):
#         def _train(self, params, dtrain, **kwargs):
#             assert dtrain.distributed
#             return super()._train(params=params, dtrain=dtrain, **kwargs)

#     train_dataset = ray.data.from_pandas(train_df)

#     trainer = DummyXGBoostTrainer(
#         scaling_config=ScalingConfig(num_workers=2),
#         label_column="target",
#         params=params,
#         datasets={TRAIN_DATASET_KEY: train_dataset},
#     )

#     assert trainer.dmatrix_params[TRAIN_DATASET_KEY]["distributed"]
#     trainer.fit()
